In [1]:
import os
import dotenv
import pymongo
import requests
import datetime
import threading
from functions import *
from flask import abort
from flask import Flask
from flask import request
from flask import session
from bson import json_util
from flask import redirect
from functools import wraps
from flask import render_template
from flask.wrappers import Response
from werkzeug.datastructures import ImmutableMultiDict

dotenv.load_dotenv()
app = Flask(__name__)
VERIFY_TOKEN = 'cashwha'
MONGO_CLIENT = pymongo.MongoClient(os.getenv('MONGO_SRV'))
WHATSAPP_CONTACTS_COL = MONGO_CLIENT["namma_yatri"]["whatsapp_contacts"]
RIDES_COL = MONGO_CLIENT["namma_yatri"]["rides"]
DRIVERS_COL = MONGO_CLIENT["namma_yatri"]["drivers"]
whatsapp_account = {
    'ACCESS_TOKEN': os.getenv('WHATSAPP_ACCESS_TOKEN'),
    'FROM_PHONE_NUMBER_ID': os.getenv('WHATSAPP_FROM_PHONE_NUMBER_ID'),
    'WABA_ID': os.getenv('WHATSAPP_WABA_ID'),
    'FROM_PHONE_NUMBER': os.getenv('WHATSAPP_FROM_PHONE_NUMBER'),
    'VERIFY_TOKEN': os.getenv('WHATSAPP_VERIFY_TOKEN')
}


In [2]:
def incoming_message(contact: dict, message: dict):
    booking_status = contact.get('booking_status', {})
    if message.get('content_type') == 'location':
        if booking_status.get('value') == 'awaiting from location':
            print("Thank you please send your to location.")
            booking_status['value'] = 'awaiting to location'
            booking_status['from'] = message['body']
            WHATSAPP_CONTACTS_COL.update_one(
                {
                    '_id': contact.get('_id')
                },
                {
                    '$set': {
                        'booking_status': booking_status
                    }
                }
            )
            return None # The user sends a from location and the server requests to send a to location
        if booking_status.get('value') == 'awaiting to location':
            print("Thank you for using Namma Yatri.\nYour ride has been scheduled and you will be notified once the ride is alotted")
            booking_status['value'] = 'ride sheduled'
            booking_status['to'] = message['body']
            WHATSAPP_CONTACTS_COL.update_one(
                {
                    '_id': contact.get('_id')
                },
                {
                    '$set': {
                        'booking_status': booking_status
                    }
                }
            )
            # make_order()
            return None # The user sends the to location and the server initiates the order
            # The order is sent back to the customer and sent to the drivers pool
        print("Kindly initiate the ride before sending your location.")
        return None # you have to initialise the order first to send your location
    if message.get('content_type') == 'text':
        message_body = message.get('body')
        if message_body == 'Book a Ride':
            print('Please Send Your current Location to Book a Ride')
            booking_status['value'] = 'awaiting from location'
            WHATSAPP_CONTACTS_COL.update_one(
                {
                    '_id': contact.get('_id')
                },
                {
                    '$set': {
                        'booking_status': booking_status
                    }
                }
            )
            return None # the server requests for the current location
        if message_body == 'Customer Care':
            print('You can contact Customer Care on call via +91 98765 43210\n\nThank you for using Namma Yatri. Have a nice day.')
            return None
        print('Hello Welcome to Namma Yatri. \nPlease select an option:\n\nBook a Ride\nCustomer Care')
        return None # The user sent a unknown message so start with a new conversation
    return None

In [3]:
contact = WHATSAPP_CONTACTS_COL.find_one({
    'from_number': '15550647406',
    'number': '919488560252'
})
message = {
    'content_type': 'text',
    'body': 'Hii'
}
# message = {
#     'content_type': 'location',
#     'body': {
#         'North': '234567890',
#         'South': '1234567890'
#     }
# }
incoming_message(contact, message)

Hello Welcome to Namma Yatri. 
Please select an option:

Book a Ride
Customer Care
